### Load the Assignment Zip file and Extract the file into Colab Locally

In [2]:
import zipfile
import os

# Specify the path to your zipped file
file_name = "/content/Assignment.zip"

# Create a directory to extract the contents into (optional)
# os.makedirs("extracted_folder", exist_ok=True)

with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall() # Extracts to the current working directory, or specify a path like "extracted_folder"

print("Folder extracted successfully!")

Folder extracted successfully!


## Preprocess, train-test split, training the BERT&SVC model

In [9]:
%cd /content/Assignment
!python src/models/train.py

/content/Assignment
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025-10-14 20:35:05.340182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760474105.360259    8628 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760474105.368327    8628 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760474105.393865    8628 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760474105.393896    8628 computation_placer.cc:177] computation placer already registered. Please check linkag

## Running the Inference on Trained Models

In [10]:
%cd /content/Assignment/src


/content/Assignment/src


## Inference with SVC Model

In [17]:
%cd /content/Assignment/src
!python infer.py \
  --model sgd \
  --model_dir /content/Assignment/experiments_sgd/run-20251014-203550/model.pkl \
  --input_json '{"title": "Login loop after MFA", "description": "User stuck on login screen after entering code"}'


/content/Assignment/src
{
  "predicted_category": "Mobile App & UI issues",
  "confidence": 0.581515,
  "raw": {
    "scores": [
      0.055740777403116226,
      0.06147587671875954,
      0.040457773953676224,
      0.04054898023605347,
      0.08594188839197159,
      0.581514835357666,
      0.0380033403635025,
      0.04846625402569771,
      0.04036979004740715,
      0.007480460684746504
    ],
    "labels": [
      "Billing and Payments",
      "Customer Service",
      "General Inquiry",
      "Human Resources",
      "IT Support",
      "Mobile App & UI issues",
      "Product Support",
      "Service Outages and Maintenance",
      "Technical Support",
      "Times & Attendance"
    ]
  }
}


## Inference with BERT model

In [19]:
!python infer.py --model bert \
                 --model_dir /content/Assignment/experiments_bert/run-20251014-203604 \
                   --input_json '{"title": "Login loop after MFA", "description": "User stuck on login screen after entering code"}'


{
  "predicted_category": "Mobile App & UI issues",
  "confidence": 0.97061,
  "raw": {
    "scores": [
      0.0006199260242283344,
      0.00048683854402042925,
      0.0004088666755706072,
      0.0009463278693147004,
      0.021751202642917633,
      0.9706102609634399,
      0.0014472976326942444,
      0.0014067336451262236,
      0.0013088842388242483,
      0.0010137417120859027
    ],
    "labels": [
      "Billing and Payments",
      "Customer Service",
      "General Inquiry",
      "Human Resources",
      "IT Support",
      "Mobile App & UI issues",
      "Product Support",
      "Service Outages and Maintenance",
      "Technical Support",
      "Times & Attendance"
    ]
  }
}


## Creating an Fast API app with Ngrok in Colab

In [23]:
%cd /content/Assignment/src


/content/Assignment/src


## Install required fastapi and ngrok packages

In [24]:
!pip install fastapi uvicorn pyngrok

## Start the FastAPI Application

In [58]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 > fastapi.log 2>&1 &


In [59]:
##ngrok authetication
!ngrok config add-authtoken api-key

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [60]:
##creating a public ngrok url

from pyngrok import ngrok

# Connect tunnel to port 8000
public_url = ngrok.connect(8000)
print("🚀 Public FastAPI URL:", public_url)


🚀 Public FastAPI URL: NgrokTunnel: "https://uncornered-ruthanne-tellurous.ngrok-free.dev" -> "http://localhost:8000"


In [64]:
##creating a tunnel between local url and ngrok and calling health check endpoint

from pyngrok import ngrok
import requests

tunnel = ngrok.connect(8000)       # returns an NgrokTunnel object
public_url = tunnel.public_url     # <-- use this string
print("Public URL:", public_url)

# 3) Call your endpoints
r = requests.get(f"{public_url}/health", timeout=10)
print("Health:", r.json())

Public URL: https://uncornered-ruthanne-tellurous.ngrok-free.dev
Health: {'status': 'ok', 'default_model_type': 'bert', 'env_model_dir': None, 'bert_latest_run': '../experiments_bert/run-20251014-203604', 'sgd_latest_run': '../experiments_sgd/run-20251014-213133/model.pkl', 'bert_loaded': True, 'bert_loaded_from': '../experiments_bert/run-20251014-203604', 'sgd_loaded': False, 'sgd_loaded_from': None}


In [65]:
## Calling /classify endpoints which takes the input of (title,description) and classifies the Support ticket into respective category

payload = {
    "title": "Clock-in not working",
    "description": "App crashes when punching in."
}
r = requests.post(f"{public_url}/classify", json=payload, timeout=20)
print(r.status_code, r.text)


200 {"predicted_category":"Mobile App & UI issues","confidence":0.8837,"raw":{"scores":[0.0028643414843827486,0.002083906438201666,0.0008071121410466731,0.002164296107366681,0.04967090114951134,0.8837218880653381,0.00782361626625061,0.003704185364767909,0.04210324585437775,0.005056548397988081],"labels":["Billing and Payments","Customer Service","General Inquiry","Human Resources","IT Support","Mobile App & UI issues","Product Support","Service Outages and Maintenance","Technical Support","Times & Attendance"],"model_type":"bert","model_dir":"../experiments_bert/run-20251014-203604"}}
